In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_auc_score
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_train_oversampled = pd.read_csv('X_train_oversampled.csv')
y_train_oversampled = pd.read_csv('y_train_oversampled.csv')
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')

# Random Forest

## 1) Standarization

In [2]:
sc = StandardScaler()
train_features = sc.fit_transform(X_train_oversampled)
train_labels = y_train_oversampled
test_features = sc.transform(X_test)
test_labels = y_test

## 2) HiperParameters Otimization

In [3]:
rf = RandomForestClassifier(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [4]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf,scoring='recall', param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_features, train_labels)

print("*******************************************")
print("RESULTS")
print("*******************************************")
print("Best Parameters:")
print(rf_random.best_params_)
print("\n*******************************************")
print("Best Estimator:")
print(rf_random.best_estimator_)
print("\n*******************************************")
print("Best Score:")
print(rf_random.best_score_)
print("\n*******************************************")


Fitting 3 folds for each of 100 candidates, totalling 300 fits


## 3) Results

In [ ]:
best_random = rf_random.best_estimator_
y_pred = best_random.predict(test_features)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))

# Balanced Random Forest

## 1) Standarization

In [ ]:
sc = StandardScaler()
train_features = sc.fit_transform(X_train)
train_labels = y_train
test_features = sc.transform(X_test)
test_labels = y_test

## 2) HiperParameters Otimization

In [ ]:
rf = BalancedRandomForestClassifier(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 3000, num = 1000)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = BalancedRandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf,scoring='recall', param_distributions = random_grid, n_iter = 300, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_features, train_labels)

print("*******************************************")
print("RESULTS")
print("*******************************************")
print("Best Parameters:")
print(rf_random.best_params_)
print("\n*******************************************")
print("Best Estimator:")
print(rf_random.best_estimator_)
print("\n*******************************************")
print("Best Score:")
print(rf_random.best_score_)
print("\n*******************************************")


## 3) Results

In [ ]:
best_random = rf_random.best_estimator_
y_pred = best_random.predict(test_features)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))

# KNN

## 1) Standarization

In [ ]:
sc = StandardScaler()
train_features = sc.fit_transform(X_train_oversampled)
train_labels = y_train_oversampled
test_features = sc.transform(X_test)
test_labels = y_test

## 2) HiperParameters Otimization

In [ ]:
knn = KNeighborsClassifier()
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(knn.get_params())

In [ ]:
# Number Neighbors
n_neighbors = [int(x) for x in np.linspace(start = 2, stop = 250, num = 249)]

random_grid = {'n_neighbors': n_neighbors}
pprint(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
knn = KNeighborsClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
knn_random = RandomizedSearchCV(estimator = knn,scoring='accuracy', param_distributions = random_grid, n_iter = 249, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
knn_random.fit(train_features, train_labels)

print("*******************************************")
print("RESULTS")
print("*******************************************")
print("Best Parameters:")
print(knn_random.best_params_)
print("\n*******************************************")
print("Best Estimator:")
print(knn_random.best_estimator_)
print("\n*******************************************")
print("Best Score:")
print(knn_random.best_score_)
print("\n*******************************************")


## 3) Results

In [ ]:
best_random = knn_random.best_estimator_
y_pred = best_random.predict(test_features)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))